# Sampling

In this notebook, picking up where we left off in the "Getting started" tutorial, we are going to walk through a round of sampling.

## Imports & Settings

First the imports.

In [1]:
import numpy as np
import scipy as sp 
from scipy.special import factorial
import pandas as pd

import emcee                      # inference and backends for sample storage
from multiprocessing import Pool  # for parallelization of the inference

import lymph

Now some settings, e.g. the name of the HDF5 file we would later like to use.

In [2]:
demo_hdf_file = "./_data/demo.hdf5"

## Set up the Model

First, we will set up the model as we would normally. In contrast to the "Getting started" notebook, we will set up a `Bilateral` model here, but that isn't more complicated. Only the data that needs to be provided to this kind of model needs to have information on the contralateral involvement as well, obviously.

In [3]:
graph = {
    ('tumor', 'primary'): ['I', 'II', 'III', 'IV'],
    ('lnl'  , 'I'):       ['II'], 
    ('lnl'  , 'II'):      ['III'],
    ('lnl'  , 'III'):     ['IV'],
    ('lnl'  , 'IV'):      []
}

In [4]:
model = lymph.Bilateral(graph)

In [5]:
diagnostic_spsn = {
    "MRI": [0.63, 0.81],
    "PET": [0.86, 0.79]
}
model.modalities = diagnostic_spsn

## Generate synthetic data

:::{note}

This step can be skipped, as that data is already in the `./_data` directory. But it may also serve as a guide on how to generate synthetic datasets.
:::

In [6]:
max_t = 10
t = np.arange(max_t + 1)

early_p = 0.3
late_p = 0.7

early_time_dist = sp.stats.binom.pmf(t, max_t, early_p)
late_time_dist = sp.stats.binom.pmf(t, max_t, late_p)
model.diag_time_dists["early"] = early_time_dist
model.diag_time_dists["late"] = late_time_dist

model.ipsi.base_probs   = [0.05, 0.2 , 0.12, 0.1 ]
model.contra.base_probs = [0.01, 0.06, 0.03, 0.01]
model.trans_probs = [0.1, 0.3, 0.2]

In [7]:
synthetic_data = model.generate_dataset(
    num_patients=200, 
    stage_dist={"early": 0.6, "late": 0.4},
)
synthetic_data.to_csv("./_data/bilateral.csv", index=False)

Now we load the data into the model instance.

In [8]:
synthetic_data = pd.read_csv("./_data/bilateral.csv", header=[0,1,2])
model.patient_data = synthetic_data

## Prepare the likelihood function

Before we can perform the sampling, we have to provide a parametric diagnose time marginalizor distribution for late T-stages. Remember that we fixed this to _generate_ the data, but during inference, we would also like to learn that parameter.

A parametric distribution over diagnose times must take two arguments: The `time_support` being the possible discrete time-steps and a parameter characterizing the distribution. In this case it is the binomial probability.

Beyond that it must raise a `ValueError` when the parameter is outside the valid range. Inside the likelihood function this error will be caught and it will return `-np.inf` to indicate to the sampler that this part of the parameter space is forbidden.

In [9]:
def binom_pmf(k: np.ndarray, n: int, p: float):
    """Binomial PMF"""
    if p > 1. or p < 0.:
        raise ValueError("Binomial prob must be btw. 0 and 1")
    q = (1. - p)
    binom_coeff = factorial(n) / (factorial(k) * factorial(n - k))
    return binom_coeff * p**k * q**(n - k)

def parametric_binom_pmf(n: int):
    """Return a parametric binomial PMF"""
    def inner(t, p):
        """Parametric binomial PMF"""
        return binom_pmf(t, n, p)
    return inner

In [10]:
model.diag_time_dists["late"] = parametric_binom_pmf(max_t)

Now we can finally define the likelihood function:

In [11]:
# plus one dimension for the late T-stage's time parameter
ndim = len(model.spread_probs) + model.diag_time_dists.num_parametric

# number of concurrent walkers that sample the space
nwalkers = 10 * ndim

# define the log-likelihood
def log_prob_fn(theta):    
    return model.likelihood(given_params=theta, log=True)

:::{admonition} Performance Warning
:class: warning

For performance reasons we have used the `model` from the outer scope, instead of passing it as a parameter to the `log_prob_fn`. This is because when parallelizing with `multiprocessing`, everything needs to be pickled to share it accross processes. And it is computationally expensive to share something as complex as our model class via pickling.
:::

## Set up a backend

`emcee` also provides us with different types of backends to store the drawn samples in. We will use its `HDFBackend`, because it stores everything in a nice HDF5 file as we go.

In [12]:
backend = emcee.backends.HDFBackend(
    filename="./_data/samples.hdf5"
)

## Sampling

:::{admonition} See also
:class: note

The creators of the `emcee` package have laid out how "sampling to convergence" works in a [really nice tutorial](https://emcee.readthedocs.io/en/stable/tutorials/monitor/). If you're serious about inference using MCMC, you should take a somewhat deep dive into this topic and estimating integrated autocorrelation times.
:::

In [13]:
# this chain will surely be too short, but it doesn't matter here
max_steps = 200

# initialize the sampler with some random samples
starting_points = np.random.uniform(size=(nwalkers,ndim))

# use Pool() from multiprocessing for parallelisation
with Pool() as pool:
    original_sampler = emcee.EnsembleSampler(
        nwalkers, ndim, log_prob_fn,
        pool=pool, backend=backend,
    )
    original_sampler.run_mcmc(initial_state=starting_points, nsteps=max_steps, progress=True)

100%|██████████| 200/200 [00:05<00:00, 35.46it/s]


We can make sure the chain of samples is actually stored by trying to retrieve it from the HDF5 file directly:

In [14]:
test_backend = emcee.backends.HDFBackend(
    filename=demo_hdf_file,
    name="original/samples",
    read_only=True
)
test_backend.shape

(120, 12)

With the stored samples one can then compute all kinds of metrics and predictions that are of interest. E.g., the _risk_ for a particular pattern of microscopic involvement, _given_ an uncertain diagnose can be computed with the model's `risk` method.